# Information content example: compute required bits

In [ ]:
%load_ext autoreload
%autoreload
import fcpy

In [ ]:
ds = fcpy.open_dataset("../data/cams_q_20191201_v3.nc")

ds = ds[["q"]].isel(lev=range(0, 10))
ds

In [ ]:
data = fcpy.sigmas_iterate_ds(
    ds, compressors=[fcpy.Round(), fcpy.LinQuantization()]
)

In [ ]:
def plot_vertical_bits(ds, y, metric):
    import matplotlib.pyplot as plt
    from cycler import cycler
    import seaborn as sns
    from matplotlib.ticker import MaxNLocator

    _, ax = plt.subplots()
    colors = sns.color_palette("colorblind", 2)
    custom_cycler = cycler(color=colors)
    ax.set_prop_cycle(custom_cycler)
    ds[metric].squeeze().plot.line(
        ax=ax, y=y, hue="compressor", marker="o", linestyle="--"
    )
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_title(f'{ds.attrs["long_name"]} in {ds.attrs["units"]}\n{metric}')
    ax.set_xlabel(metric)
    ax.set_ylabel("Model level")

In [ ]:
for data_ in data.values():
    plot_vertical_bits(data_, y="lev", metric="bits")

In [ ]:
for data_ in data.values():
    plot_vertical_bits(data_, y="lev", metric="sigmas")

In [ ]:
for data_ in data.values():
    plot_vertical_bits(data_, y="lev", metric="bitinf")

In [ ]:
# Here we compare the spatial change of specific humidity
# by rounding values to 8 bits
fcpy.spatialplot(
    ds=ds,
    baseline=fcpy.Float(bits=32),
    var_name="q",
    compressor=fcpy.Round(bits=6),
    metric=fcpy.Difference,
    latitude=0,
    longitude=0,
    lev=1,
    third_dim="lev",
)